In [2]:
import pandas as pd
from Utils.DatasetClass import SubmissionDataset
import torchvision
from glob import glob
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

## Prepping the data

In [3]:
test_file_path = "./sample_submission.csv"

relationships_temp = pd.read_csv(test_file_path)
relationships = []

for x in relationships_temp['img_pair']:
    relationships.append(x.split('-'))

In [4]:
test_images_path = "./test/"

all_images = glob(test_images_path + "*.jpg")

test_person_to_images = {}

for x in all_images:

    if x.split("\\")[-1] not in test_person_to_images:
        test_person_to_images[x.split("\\")[-1]] = [x]

    else:
        test_person_to_images[x.split("\\")[-1]].append(x)

test_dataset = SubmissionDataset(relations = relationships, person_to_image= test_person_to_images)
testloader = DataLoader(test_dataset, batch_size= 1)

## Submission Testing Function definition

In [5]:
def submission_testing(model, testloader, device):
    model.eval()
    model.to(device)
    pred_list = []
    
    for batch in testloader:
        tensor1, tensor2 = batch
        tensor1, tensor2 = tensor1.to(device), tensor2.to(device)
        with torch.no_grad():
            output = model(tensor1, tensor2)
            pred = output>0.5
        pred_list.append(pred)

    return pred_list

## Instantiate model

In [6]:
from Utils.SiameseNet import SiameseNet
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_choice = 'test_epoch131.pt'
MODEL_PATH = os.getcwd() + '\\models\\'+model_choice

model = SiameseNet()
model.load_state_dict(torch.load(MODEL_PATH, map_location= device))
model.eval()

c:\Users\ianch\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SiameseNet(
  (encoder): InceptionResnetV1(
    (conv2d_1a): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2a): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2b): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_3b): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, t

## Prediction starts

In [7]:
pred_list = submission_testing(model, testloader, device)

In [11]:
new_list = []

for y in pred_list:
    if y == False:
        new_list.append(0)
    else:
        new_list.append(1)

## Export Submission file

In [15]:
test_file_path = "./sample_submission.csv"

df = pd.read_csv(test_file_path)
df['is_related'] = new_list

output_file_name = 'csvSubmissions/final_submission.csv'

df.to_csv(output_file_name, index=False)